# Homework 3 Group 9
## Michael Karaman, Alexys Lamkin, Yiyun Zhang

1. Implement a Spam Classifier using MapReduce (MRJob)

In [1]:
!pip install mrjob

     |████████████████████████████████| 439 kB 5.3 MB/s 


In [2]:
import pandas as pd
import requests
import io


url = "https://cdn.discordapp.com/attachments/859295073329741857/881393974907125810/spam.csv"
res = requests.get(url)
file = io.StringIO(res.content.decode('ISO-8859-1'))
df = pd.read_csv(file)
df = df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'])
length = df.shape[0]
trainLength = int(0.8*length)
testLength = length-trainLength-1
dfTrain = df[0:trainLength]
dfTest = df[trainLength+1:]
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [3]:
counter = 0
f = open("spamTrain.txt", "w")
while counter<trainLength:
  line = dfTrain.v1[counter]+" "+dfTrain.v2[counter]
  f.write(line+"\n")
  counter += 1
f.close()

In [4]:
counter += 1
f = open("spamTest.txt", "w")
while counter<length:
  line = dfTest.v1[counter]+" "+dfTest.v2[counter]
  f.write(line+"\n")
  counter += 1
f.close()

In [5]:
%%file spamProb.py
from mrjob.job import MRJob
import re
WORD_RE = re.compile(r"[\w']+")

class spamProb(MRJob):
  def mapper(self, _, line):
    firstWord = line.split()[0]
    line = line.split(' ', 1)[1]
    if firstWord == "spam":
      for word in WORD_RE.findall(line):
        yield word.lower(), (1,1)
    else:
      for word in WORD_RE.findall(line):
        yield word.lower(), (0,1)

        
  def reducer(self, word, counts):
    sorh = 0
    counter = 0
    for spam, count in counts:
      sorh += spam
      counter += count
    yield word, (sorh/counter)

if __name__ == '__main__':
    spamProb.run()

Writing spamProb.py


In [6]:
!python spamProb.py spamTrain.txt > spamProb.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/spamProb.root.20210905.163140.823252
Running step 1 of 1...
job output is in /tmp/spamProb.root.20210905.163140.823252/output
Streaming final output from /tmp/spamProb.root.20210905.163140.823252/output...
Removing temp directory /tmp/spamProb.root.20210905.163140.823252...


In [7]:
spamProbList = []
f = open("spamProb.txt", "r")
for line in f:
  probList = [line.split()[0].strip('"'),line.split()[1].strip('"')]
  spamProbList.append(probList)
f.close()

In [8]:
def getSpamProb(key):
  for spamList in spamProbList:
    if(spamList[0] == key):
      return float(spamList[1])

In [9]:
import re
WORD_RE = re.compile(r"[\w']+")
f = open("spamTest.txt", "r")
success = 0
for line in f:
  firstWord = line.split()[0]
  line = line.split(' ', 1)[1]
  pSpam = 1.0
  pHam = 1.0
  for word in WORD_RE.findall(line):
    spamP = getSpamProb(word)
    if(spamP != None):
      pSpam *= spamP
      pHam *= (1-spamP)
  if pSpam > pHam and firstWord == "spam":
    success += 1
  if pSpam < pHam and firstWord == "ham":
    success += 1
f.close()
print("Accuracy: ", success/testLength)

Accuracy:  0.9093357271095153


2. Implement a Spam Classifier using PySpark

In [10]:
#installing pyspark
!pip install -q pyspark

     |████████████████████████████████| 212.4 MB 64 kB/s 
     |████████████████████████████████| 198 kB 47.6 MB/s 


In [11]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName('hw3').setMaster('local')
sc = SparkContext(conf=conf)

In [12]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import NaiveBayes
import re

WORD_RE = re.compile(r"[\w']+")
f = open("spamTrain.txt", "r")
words = {}
data = []

for line in f:
  label = line.split()[0]

  if (len(line.split(' ', 1)) > 1):
    email = line.split(' ', 1)[1]
 
    for word in WORD_RE.findall(email):
      if word not in words:
        words[word] = 1
      else:
        words[word] += 1
f.close()

In [ ]:
f = open("spamTrain.txt", "r")
for line in f: 
    label = line.split()[0]

    if (len(line.split(' ', 1)) > 1):
        email = line.split(' ', 1)[1].split(' ')
        probs = []
        for word in words:
            if word in email:
                probs.append(words[word]/sum(words.values()))
            else:   
                probs.append(0)
        if label == 'spam':
            data.append(LabeledPoint(1.0, probs))
        elif label == 'ham':
            data.append(LabeledPoint(0.0, probs))

f.close()

dataP = sc.parallelize(data)

model = NaiveBayes.train(dataP, 1.0)

In [ ]:
f = open("spamTest.txt", "r")
actual = []
predicted= []
correct = 0
for line in f: 
    label = line.split()[0]

    if label == 'spam':
      actual.append(1.0)
    elif label == "ham":
      actual.append(0.0)

    if (len(line.split(' ', 1)) > 1):
        email = line.split(' ', 1)[1].split(' ')
        probs = []
        for word in words:
            if word in email:
                probs.append(words[word]/sum(words.values()))
            else:   
                probs.append(0)
        prediction = model.predict(probs)
        predicted.append(prediction)
f.close()

for i, prediction in enumerate(predicted):
    if predicted[i] == actual[i]:
      correct += 1

print("accuracy: " str(correct/len(actual)))


3. Conclusion

Using MRJob and Pyspark provided quick and accurate classifers both achiveing 90% accuracy. This is quite impressive given that the dataset didn't require much cleaning. We believe with more cleanig, the accuracy could be improved. 